In [1]:
import string
from east.asts import base
from east import utils
import glob
import errno
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from multiprocessing.dummy import Pool
import tqdm
import pickle
from itertools import product


from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /home/deth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def clear_sentence(sentence):
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    stripped = sentence.translate(table)
    tokens = word_tokenize(stripped)
    tokens = [w.lower() for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    porter = PorterStemmer()
    wnl = WordNetLemmatizer()
    lemmatized = [wnl.lemmatize(word) for word in words]
    stemmed = [porter.stem(word) for word in words]
    return lemmatized

In [3]:
def text_substrings(text, k):
    substrings_array = []
    for i in range(len(text) - k + 1):
        substring = " ".join(text[i : i + k])
        substrings_array.append(substring)
        i+=1
    return substrings_array

In [4]:
def calculate(text, leafs):
    new = []
    if (len(text)) != 0:
        ast = base.AST.get_ast(text)
        # Compute the relevance of a keyphrase to the text collection indexed by this AST.
        # The relevance score will always be in [0; 1]
        for j in range(len(leafs)):
            try:
                score = ast.score(leafs[j])
                new.append(score)
            except ZeroDivisionError:
                print("problem\n",text, '\n', leafs[j])
    else:
        new = np.zeros(len(leafs))
    return new


In [11]:
filenames = sorted(glob.glob('./Data/*.csv'))
    
li = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [6]:
# read lists of requirements,responsibiliries, conditions from files
f = open('./resp_file.txt', 'r')
responsibilities = f.read().split('\n')
f = open('./req_file.txt', 'r')
requirements = f.read().split('\n')
f.close()
f = open('./workenv_file.txt', 'r')
conditions = f.read().split('\n')
f.close()
requirements = requirements[:-1]
conditions = conditions[:-1]
responsibilities = responsibilities[:-1]

In [7]:
from progressbar import Percentage, ProgressBar,Bar,ETA

In [8]:
someMaxValue=len(requirements)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)
clear_req = []
for sent in pbar(requirements):
    cleared = ' '.join(clear_sentence(sent))
    if cleared != '':
        clear_req.append(cleared)
    else:
        clear_req.append(sent)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 0:00:01


In [9]:
someMaxValue=len(responsibilities)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)
clear_resp = []
for sent in pbar(responsibilities):
    cleared = ' '.join(clear_sentence(sent))
    if cleared != '':
        clear_resp.append(cleared)
    else:
        clear_resp.append(sent)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 0:00:00


In [8]:
someMaxValue=len(conditions)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)
clear_cond = []
for sent in pbar(conditions):
    cleared = ' '.join(clear_sentence(sent))
    if cleared != '':
        clear_cond.append(cleared)
    else:
        clear_cond.append(sent)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 0:00:01


In [12]:
texts = frame['Job Description']

someMaxValue=len(texts)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)

strings_collection = []

for text in pbar(texts):
    strings_collection.append(text_substrings(clear_sentence(text), 7))

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 0:01:43


In [36]:
req_count = []
someMaxValue=len(strings_collection)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)
for strings in pbar(strings_collection):
    req_count.append(calculate(strings, clear_req))

df_req = pd.DataFrame(req_count, index=np.arange(len(texts)), columns=requirements)
df_req.to_csv("./Data/Result_req.csv")
del df_req
del req_count

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 5:24:59


In [ ]:
resp_count = []
someMaxValue=len(strings_collection)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)
for strings in pbar(strings_collection):
    resp_count.append(calculate(strings, clear_resp))

df_resp = pd.DataFrame(resp_count, index=np.arange(len(texts)), columns=responsibilities)
df_resp.to_csv("./Data/Result_resp.csv")
del df_resp
del resp_count

In [14]:
cond_count = []
someMaxValue=len(strings_collection)
pbar = ProgressBar(widgets=[Bar('>', '[', ']'), ' ',
                                            Percentage(), ' ',
                                            ETA()],maxval=someMaxValue)

for strings in pbar(strings_collection):
#     req_count.append(calculate(strings, clear_req))
#     resp_count.append(calculate(strings, clear_resp))
    cond_count.append(calculate(strings, clear_cond))
df_count = pd.DataFrame(cond_count, index=np.arange(len(texts)), columns=conditions)
df_count.to_csv("./Result_we.csv")
del df_count
del cound_count
# df_req.to_csv("./Data/Result_req.csv")
# df_resp.to_csv("./Data/Result_resp.csv")
# df_count.to_csv("./Data/Result_count.csv")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100% Time: 2:30:05


NameError: name 'cound_count' is not defined